# LLM - Evauation

Credit: Coursera

In [1]:
import os
import json
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from IPython.display import display, Markdown

In [2]:
with open('../configs/config.json', 'r') as f:
    config = json.load(f)

In [3]:
os.environ['OPENAI_API_TYPE']=config['OPENAI_API_TYPE']
os.environ['OPENAI_API_VERSION']=config['OPENAI_API_VERSION']
os.environ['OPENAI_API_BASE']=config['OPENAI_API_BASE']
os.environ['OPENAI_API_KEY']=config['OPENAI_API_KEY']

In [13]:
file = '../data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [16]:
pd.read_csv(file, index_col=0).head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


### Define LLM

In [6]:
llm = AzureChatOpenAI(deployment_name=config['OPENAI_MODEL_NAME'], 
                       model_name="gpt-35-turbo", 
                       temperature=0.0)

In [7]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002',
                              deployment='embeddings',
                              openai_api_base=config['OPENAI_API_BASE'],
                              openai_api_type=config['OPENAI_API_TYPE'],
                              openai_api_key=config['OPENAI_API_KEY'],
                              chunk_size=1)
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

In [11]:
qna = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Preparing test datapoints

In [22]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': '../data/OutdoorClothingCatalog_1000.csv', 'row': 10})

In [23]:
data[11]

Document(page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': '../data/OutdoorClothingCatalog_1000.csv', 'row': 11})

In [24]:
test_examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated test data points

In [25]:
from langchain.evaluation.qa import QAGenerateChain

In [27]:
test_gen_chain = QAGenerateChain.from_llm(llm)

In [28]:
new_examples = test_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [30]:
new_examples[0]

{'query': "What is the weight of one pair of Women's Campside Oxfords?",
 'answer': "The approximate weight of one pair of Women's Campside Oxfords is 1 lb. 1 oz."}

In [31]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': '../data/OutdoorClothingCatalog_1000.csv', 'row': 0})

### Combine examples

In [32]:
test_examples += new_examples

In [34]:
qna.run(test_examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'The Cozy Comfort Pullover Set, Stripe has side pockets.'

## Manual Evalution

In [37]:
import langchain

In [38]:
langchain.debug = True
qna.run(test_examples[0]["query"])
langchain.debug = False

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded h

## LLM assisted evaluation

In [39]:
predictions = qna.apply(test_examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [40]:
from langchain.evaluation.qa import QAEvalChain

In [42]:
eval_chain = QAEvalChain.from_llm(llm) # Note: Make sure temperature is set to zero here!

In [43]:
graded_outputs = eval_chain.evaluate(test_examples, predictions)

In [45]:
for i, eg in enumerate(test_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: The Cozy Comfort Pullover Set, Stripe has side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What is the weight of one pair of Women's Campside Oxfords?
Real Answer: The approximate weight of one pair of Women's Campside Oxfords is 1 lb. 1 oz.
Predicted Answer: The weight of one pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.
Predicted Grade: CORRECT

Example 3:
Question: What are the dimensions of the small and medium Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The small Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28" and the medium has dimensions of 22.5" x 34.